## 主要函数说明
>卷积层：
tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)

### 参数说明
* data_format：表示输入的格式，有两种分别为：“NHWC”和“NCHW”，默认为“NHWC”
* input：输入是一个4维格式的（图像）数据，数据的 shape 由 data_format 决定：当 data_format 为“NHWC”输入数据的shape表示为[batch, in_height, in_width, in_channels]，分别表示训练时一个batch的图片数量、图片高度、 图片宽度、 图像通道数。当 data_format 为“NCWH”输入数据的shape表示为[batch, in_channels， in_height, in_width]
* filter：卷积核是一个4维格式的数据：shape表示为：[height,width,in_channels, out_channels]，分别表示卷积核的高、宽、深度（与输入的in_channels应相同）、输出 feature map的个数（即卷积核的个数）。
* strides：表示步长：一个长度为4的一维列表，每个元素跟data_format互相对应，表示在data_format每一维上的移动步长。当输入的默认格式为：“NHWC”，则 strides = [batch , in_height , in_width, in_channels]。其中 batch 和 in_channels 要求一定为1，即只能在一个样本的一个通道上的特征图上进行移动，in_height , in_width表示卷积核在特征图的高度和宽度上移动的布长，即 strideheight 和 stridewidth 。
* padding：表示填充方式：“SAME”表示采用填充的方式，简单地理解为以0填充边缘，当stride为1时，输入和输出的维度相同；“VALID”表示采用不填充的方式，多余地进行丢弃。具体公式：

>池化层：
tf.nn.max_pool( value, ksize,strides,padding,data_format=’NHWC’,name=None) 

### 参数说明
* value: 表示池化的输入：一个4维格式的数据，数据的 shape 由 data_format 决定，默认情况下shape 为[batch, height, width, channels]
* 其他参数与 tf.nn.cov2d 类型
* ksize：表示池化窗口的大小：一个长度为4的一维列表，一般为[1, height, width, 1]，因不想在batch和channels上做池化，则将其值设为1。

>Batch Nomalization层
batch_normalization( x,mean,variance,offset,scale, variance_epsilon,name=None)

* mean 和 variance 通过 tf.nn.moments 来进行计算： batch_mean, batch_var = tf.nn.moments(x, axes = [0, 1, 2], keep_dims=True)，注意axes的输入。对于以feature map 为维度的全局归一化，若feature map 的shape 为[batch, height, width, depth]，则将axes赋值为[0, 1, 2]
* x 为输入的feature map 四维数据，offset、scale为一维Tensor数据，shape 等于 feature map 的深度depth。

## 通过搭建卷积神经网络来实现sklearn库中的手写数字识别

### 搭建的卷积神经网络结构如图

![](http://img.blog.csdn.net/20170501131714709?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvY3htc2Ni/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)



In [2]:
import tensorflow as tf
from sklearn.datasets import load_digits
import numpy as np

In [3]:
digits = load_digits()

In [4]:
digits

{'DESCR': "Optical Recognition of Handwritten Digits Data Set\n===================================================\n\nNotes\n-----\nData Set Characteristics:\n    :Number of Instances: 5620\n    :Number of Attributes: 64\n    :Attribute Information: 8x8 image of integer pixels in the range 0..16.\n    :Missing Attribute Values: None\n    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)\n    :Date: July; 1998\n\nThis is a copy of the test set of the UCI ML hand-written digits datasets\nhttp://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits\n\nThe data set contains images of hand-written digits: 10 classes where\neach class refers to a digit.\n\nPreprocessing programs made available by NIST were used to extract\nnormalized bitmaps of handwritten digits from a preprinted form. From a\ntotal of 43 people, 30 contributed to the training set and different 13\nto the test set. 32x32 bitmaps are divided into nonoverlapping blocks of\n4x4 and the number of on pixels ar

In [5]:
X_data = digits.data.astype(np.float32)
Y_data = digits.target.astype(np.float32).reshape(-1, 1)

In [6]:
X_data.shape

(1797, 64)

In [7]:
Y_data.shape

(1797, 1)

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_data = scaler.fit_transform(X_data)

In [9]:
from sklearn.preprocessing import OneHotEncoder
Y = OneHotEncoder().fit_transform(Y_data).todense()

In [10]:
Y

matrix([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 1., 0.]])

In [11]:
# 转换成图片的格式　(batch, height, width, channels)
X = X_data.reshape(-1, 8, 8, 1)
batch_size = 8 # 使用MBGD算法，设定batch_size=8

In [12]:
def generatebatch(X, Y, n_examples, batch_size):
    for batch_i in range(n_examples // batch_size):
        start = batch_i * batch_size
        end = start + batch_size
        batch_xs = X[start:end]
        batch_ys = Y[start:end]
        yield batch_xs, batch_ys

In [13]:
tf.reset_default_graph()
# 输入层
tf_X = tf.placeholder(tf.float32, [None, 8, 8, 1])
tf_Y = tf.placeholder(tf.float32, [None, 10])

In [15]:
# 卷积层＋激活层
conv_filter_w1 = tf.Variable(tf.random_normal([3, 3, 1, 10]))
conv_filter_b1 = tf.Variable(tf.random_normal([10]))
relu_feature_maps1 = tf.nn.relu(tf.nn.conv2d(tf_X, conv_filter_w1, strides=[1,1,1,1],padding='SAME') + conv_filter_b1)


In [16]:
# 池化层
max_pool1 = tf.nn.max_pool(relu_feature_maps1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

In [17]:
max_pool1

<tf.Tensor 'MaxPool:0' shape=(?, 4, 4, 10) dtype=float32>

In [18]:
# 卷积层
conv_filter_w2 = tf.Variable(tf.random_normal([3, 3, 10, 5]))
conv_filter_b2 = tf.Variable(tf.random_normal([5]))
conv_out2 = tf.nn.conv2d(relu_feature_maps1, conv_filter_w2, strides=[1, 2, 2, 1],padding='SAME') + conv_filter_b2

In [19]:
conv_out2

<tf.Tensor 'add_1:0' shape=(?, 4, 4, 5) dtype=float32>

In [21]:
# BN归一化层　＋ 激活层
batch_mean, batch_var = tf.nn.moments(conv_out2, [0, 1, 2], keep_dims=True)
shift = tf.Variable(tf.zeros([5]))
scale = tf.Variable(tf.ones([5]))
epsilon = 1e-3
BN_out = tf.nn.batch_normalization(conv_out2, batch_mean, batch_var, shift, scale, epsilon)

In [22]:
BN_out

<tf.Tensor 'batchnorm/add_1:0' shape=(?, 4, 4, 5) dtype=float32>

In [23]:
relu_BN_maps = tf.nn.relu(BN_out)

In [24]:
# 池化层
max_pool2 = tf.nn.max_pool(relu_BN_maps, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

In [25]:
max_pool2

<tf.Tensor 'MaxPool_1:0' shape=(?, 2, 2, 5) dtype=float32>

In [26]:
# 将特征图进行展开
max_pool2_flat = tf.reshape(max_pool2, [-1, 2*2*5])

In [28]:
# 全连接层
fc_w1 = tf.Variable(tf.random_normal([2*2*5, 50]))
fc_b1 = tf.Variable(tf.random_normal([50]))
fc_out1 = tf.nn.relu(tf.matmul(max_pool2_flat, fc_w1) + fc_b1)

In [29]:
# 输出层
out_w1 = tf.Variable(tf.random_normal([50, 10]))
out_b1 = tf.Variable(tf.random_normal([10]))
pred = tf.nn.softmax(tf.matmul(fc_out1, out_w1) + out_b1)

In [30]:
loss = -tf.reduce_mean(tf_Y*tf.log(tf.clip_by_value(pred, 1e-11, 1.0)))

In [31]:
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [32]:
y_pred = tf.arg_max(pred, 1)
bool_pred = tf.equal(tf.arg_max(tf_Y, 1), y_pred)

In [33]:
accuracy = tf.reduce_mean(tf.cast(bool_pred, tf.float32)) # 准确率

In [34]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1000):
        for batch_xs, batch_ys in generatebatch(X, Y, Y.shape[0], batch_size):
            sess.run(train_step, feed_dict={tf_X:batch_xs, tf_Y:batch_ys})
        if(epoch%100==0):
            res = sess.run(accuracy, feed_dict={tf_X:X, tf_Y:Y})
            print(epoch, res)
    res_ypred = y_pred.eval(feed_dict={tf_X:X, tf_Y:Y}).flatten()
    print(res_ypred)

0 0.25431275
100 0.9682804
200 0.99499154
300 0.99944353
400 0.99944353
500 1.0
600 1.0
700 1.0
800 0.9983306
900 0.99888706
[0 1 2 ... 8 9 8]


在第100次个batch size迭代时，准确率就快速收敛了， 这得归功于Batch Normaliztion的作用，需要注意的是，这个模型还不能用来预测单个样本，因为在进行BN层计算时，单个样本的均值和方差都为0，会得到相反的预测效果

In [35]:
from sklearn.metrics import accuracy_score

In [36]:
print(accuracy_score(Y_data, res_ypred.reshape(-1, 1)))

0.9994435169727324
